In [1]:
pip install textblob

     |████████████████████████████████| 636 kB 890 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [162]:
import pandas as pd
#from datascience import *
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
from textblob import TextBlob
import twint
%matplotlib inline

In [163]:
import nltk
from nltk.stem import *
import re 

## Load in the data:

## Add a polarity column:

In [164]:
def get_polarity(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

## Preprocess the data:

#### Create a preprocessor function to preprocess the tweets & make a new column with the preprocessed text:

In [165]:
def preprocessor(data):
    
    # TfidfVectorizer puts all letters in lowercase by default
    data = data.lower()

    # sub hyphens with no space
    data = re.sub("\-", '', data)
    
    # sub non letter & non digit characters w/ a space
    data = re.sub("[^A-Za-z0-9 \\n]"," ", data)
    
    # sub digit characters with '#' character
    data = re.sub("[\d]", "\#", data) 
    
    # removing stop words
    stop_words = ['and', 'but', 'the', 'just', 'or', 'it', 'of', 'to', 'her','he']
    for word in stop_words:
        data = data.replace(word, "")
    
    # stemming words
    stem = PorterStemmer()
    for word in data:
        data = data.replace(word, stem.stem(word))
        
    # shorten long words of length 7 or more
    data = re.sub('(\w{7})\w+', '\\1', data)
    
    return data

#### Featurize the data with TfidfVectorizer & add feature names to the dataframe:

In [166]:
def tvect(data):
    vect = TfidfVectorizer(min_df = 20)
    fitted = vect.fit_transform(data)
    return fitted, vect.get_feature_names()

featurized = tvect(df.preprocess_full_text)

## MAKE INTO 1 BIG FUNCTION:

In [167]:
def featurize(csv, path):
    #read in csv & make a df
    df = pd.read_csv(path+ csv)
    
    df = df.rename(columns={'id': 'tweet_id'})
    #df = df.set_index('tweet_id')
    
    #add polarity column
    df['polarity'] = df['full_text'].apply(get_polarity)
    
    #add preprocessed column
    df['preprocess_full_text'] = df.full_text.apply(lambda x: preprocessor(x))
    
    #add feature names from TfidfVectorizer
    featurized = tvect(df.preprocess_full_text)
    df1 = pd.DataFrame(featurized[0].toarray(), columns=featurized[1])
    df = pd.concat([df, df1], axis=1)
    
    return df

In [168]:
df = featurize( 'tweet_mobility.csv' , '../data/')

 ## Add in the sentiment score:

In [169]:
s = pd.read_csv('../data/sentiment_scores.csv')
s = s.rename(columns={'id': 'tweet_id'})

In [170]:
df = df.merge(s, on = 'tweet_id', how = 'left')

In [171]:
df = df.iloc[727:, :]

In [172]:
df = df.drop('Unnamed: 0', axis = 1)
df

,date,full_text,geo,truncated,display_text_range,user,tweet_id,entities,possibly_sensitive,longitude,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
727,2020-03-30,Definitely making a routine to stop from going...,"[41.42362498, -122.39264801]",False,"[0, 239]","{'id': 988896150571790336, 'id_str': '98889615...",1244710975422574592,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.392648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.062500
728,2020-03-30,This is my 'self-isolating' face. #quarantine ...,"[34.1722, -118.378]",False,"[0, 140]","{'id': 1008423932267921408, 'id_str': '1008423...",1244710063589089281,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.378000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
729,2020-03-30,Corona virus testing and treatment is availabl...,"[33.97977, -118.18884]",False,"[0, 233]","{'id': 107183997, 'id_str': '107183997', 'name...",1244708953994469377,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.188840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000
730,2020-03-30,"""A box filled with human shit"" (Corona Heights...","[37.7622607, -122.4435337]",False,"[0, 75]","{'id': 1104150794914344960, 'id_str': '1104150...",1244679522588352514,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.443534,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667
731,2020-03-30,‘I’m not going out there.” #quarantine #corona...,"[34.1467, -118.1488]",False,"[0, 109]","{'id': 131266206, 'id_str': '131266206', 'name...",1244697670947500032,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.148800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18253,2020-09-20,"Quarantine project done\n💌 @ Emeryville, Calif...","[37.83138889, -122.28527778]",True,"[0, 74]","{'id': 6072742, 'id_str': '6072742', 'name': '...",1307590494843629570,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.285278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
18254,2020-09-20,#Collaborations during #covid #web #cams and T...,"[34.45139143, -118.55102682]",True,"[0, 235]","{'id': 777970362134859776, 'id_str': '77797036...",1307616765594218496,"{'hashtags': [{'text': 'Collaborations', 'indi...",False,-118.551027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300000
18255,2020-09-20,Vaccine Awareness Week Update 2020 https://t.c...,"[37.00748787, -121.5705372]",True,"[0, 58]","{'id': 765682887899226112, 'id_str': '76568288...",1307622540274069505,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-121.570537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
18256,2020-09-20,Month 6 spa update. \nPlease Dm or email with ...,"[37.8029, -122.2721]",True,"[0, 238]","{'id': 4477075459, 'id_str': '4477075459', 'na...",1307523162079330307,"{'hashtags': [{'text': 'covid19', 'indices': [...",False,-122.272100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [173]:
df.to_csv("../data/final_data.csv", index = False)

In [174]:
df = pd.read_csv('../data/final_data.csv')
df

,date,full_text,geo,truncated,display_text_range,user,tweet_id,entities,possibly_sensitive,longitude,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
0,2020-03-30,Definitely making a routine to stop from going...,"[41.42362498, -122.39264801]",False,"[0, 239]","{'id': 988896150571790336, 'id_str': '98889615...",1244710975422574592,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.392648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.062500
1,2020-03-30,This is my 'self-isolating' face. #quarantine ...,"[34.1722, -118.378]",False,"[0, 140]","{'id': 1008423932267921408, 'id_str': '1008423...",1244710063589089281,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.378000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2020-03-30,Corona virus testing and treatment is availabl...,"[33.97977, -118.18884]",False,"[0, 233]","{'id': 107183997, 'id_str': '107183997', 'name...",1244708953994469377,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.188840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400000
3,2020-03-30,"""A box filled with human shit"" (Corona Heights...","[37.7622607, -122.4435337]",False,"[0, 75]","{'id': 1104150794914344960, 'id_str': '1104150...",1244679522588352514,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.443534,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066667
4,2020-03-30,‘I’m not going out there.” #quarantine #corona...,"[34.1467, -118.1488]",False,"[0, 109]","{'id': 131266206, 'id_str': '131266206', 'name...",1244697670947500032,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.148800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17526,2020-09-20,"Quarantine project done\n💌 @ Emeryville, Calif...","[37.83138889, -122.28527778]",True,"[0, 74]","{'id': 6072742, 'id_str': '6072742', 'name': '...",1307590494843629570,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.285278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
17527,2020-09-20,#Collaborations during #covid #web #cams and T...,"[34.45139143, -118.55102682]",True,"[0, 235]","{'id': 777970362134859776, 'id_str': '77797036...",1307616765594218496,"{'hashtags': [{'text': 'Collaborations', 'indi...",False,-118.551027,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.300000
17528,2020-09-20,Vaccine Awareness Week Update 2020 https://t.c...,"[37.00748787, -121.5705372]",True,"[0, 58]","{'id': 765682887899226112, 'id_str': '76568288...",1307622540274069505,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-121.570537,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
17529,2020-09-20,Month 6 spa update. \nPlease Dm or email with ...,"[37.8029, -122.2721]",True,"[0, 238]","{'id': 4477075459, 'id_str': '4477075459', 'na...",1307523162079330307,"{'hashtags': [{'text': 'covid19', 'indices': [...",False,-122.272100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
